In [ ]:
import numpy as np
import pandas as pd
import sklearn
import cython
import talib
import bokeh


from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from Functions import DataReady, KijunSen, GetData

In [ ]:
#GetData

In [ ]:
#data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min.csv")
data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min(240831).csv")

DataReady(data)

In [4]:
class MachineLearningStrategyI1000(Strategy):
    
    # Add variables to the strategy
    atr_lenght = 5
    times_atr_tp = 10
    times_atr_sl = 12
    upper_ema_lenght = 67
    lower_ema_lenght = 199
    stoch_rsi_lenght = 27
    kijun_sen_period = 19
    kijun_sen_price_range = 4

    def init(self):
        # Indicator code
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_lenght)
        self.ema_upper = self.I(talib.EMA, self.data.Close, self.upper_ema_lenght)
        self.ema_lower = self.I(talib.EMA, self.data.Close, self.lower_ema_lenght)
        self.stoch_rsi = self.I(talib.STOCHRSI, self.data.Close, self.stoch_rsi_lenght)
        self.kijun_sen = self.I(KijunSen, data, self.kijun_sen_period)
        self.kijun_sen_to_price = self.data.Close - self.kijun_sen

        
    def next(self):
        if crossover(self.ema_lower, self.ema_upper) and self.kijun_sen_to_price < self.kijun_sen_price_range / 10:
            self.buy(sl = self.data.Close - (self.atr*self.times_atr_sl / 10), tp = self.data.Close + (self.atr*self.times_atr_tp / 10))
        

    
bt = Backtest(data, MachineLearningStrategyI1000, cash = 10000, hedging = True)

stats = bt.optimize(
    random_state = 1,
    max_tries = 1000,
    return_heatmap = True,
    method = "skopt",
    upper_ema_lenght = range(20, 100, 1),
    lower_ema_lenght = range(100, 200, 1),
    kijun_sen_period = range(10, 40, 1),
    times_atr_tp = range(10, 30, 1),
    times_atr_sl = range(10, 30, 1),
    atr_lenght = range(5, 30, 1),
    stoch_rsi_lenght = range(5, 40, 1),
    kijun_sen_price_range = range(0, 10, 1),
    maximize = "Sharpe Ratio"
)

print(stats)
#bt.run()

bt.plot()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x702439e24830>>
Traceback (most recent call last):
  File "/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
